In [286]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.distance import geodesic
from shapely.geometry import Point, LineString

In [325]:
# Read the data from the csv file
airport_info = pd.read_csv('Assignment_Data/Group_4_Airport_info.csv')

# We make a dictionary with the airport code as the key the rest of the columns as sub-dictionaries
airports = {}

for index, row in airport_info.iterrows():
    airports[row['ICAO Code']] = row.to_dict()

In [288]:
# Create distance matrix
# Create a list of all the airports
airport_list = airport_info['ICAO Code'].tolist()

# Create a dictionary of all the airports and their coordinates
airport_coords = {}

for i in airport_list:
    airport_coords[i] = (airport_info[airport_info['ICAO Code'] == i]['Latitude (deg)'].values[0],
                       airport_info[airport_info['ICAO Code'] == i]['Longitude (deg)'].values[0])

# Create a distance matrix
distance = {}

# Fill the distance matrix in a dict with the distances between the airports
for i in airport_list:
    distance[i] = {}
    for j in airport_list:
        distance[i][j] = geodesic(airport_coords[i], airport_coords[j]).km

In [289]:
# Import aircraft info
aircraft_info = pd.read_csv('Assignment_Data/Aircraft_info.csv')

# Add a column for BT (Block Time) to the aircraft_info dataframe
aircraft_info['BT'] = 10  # [hours]

# Create a dictionary with the aircraft types as keys and the rest of the columns as sub-dictionaries
aircrafts = {}
for index, row in aircraft_info.iterrows():
    aircrafts[row['AC_type']] = row.to_dict()

# Rename the main keys in the aircraft dictionary to a numeric value with format AC_#

# Create a list of the keys in the aircraft dictionary
keys = list(aircrafts.keys())

# Create a new dictionary with the new keys
new_keys = {}
for i in range(len(keys)):
    new_keys[keys[i]] = 'AC_' + str(i+1)

aircraft_types = list(new_keys.values())

# Create a new dictionary with the new keys
aircrafts_new = {}
for i in range(len(keys)):
    aircrafts_new['AC_' + str(i+1)] = aircrafts[keys[i]]

# Replace the keys in the aircraft dictionary with the new keys
aircrafts = aircrafts_new


In [290]:
# Import demand info
demand_info = pd.read_csv('Assignment_Data/Group_4_Demand.csv')
demand_info.rename(columns={'Unnamed: 0': 'Origin'}, inplace=True)

# Create a dictionary with the demand info
demand = {}
for index, row in demand_info.iterrows():
    demand[row['Origin']] = row.to_dict()

# Remove the origin column from the demand
for i in demand:
    demand[i].pop('Origin', None)

In [291]:
# Make LIRF the hub with a dictionary of all the airports with a 1 if it is the hub and 0 if it is not
hub = "LIRF"

g = {}
for i in airport_list:
    if i == hub:
        g[i] = 0
    else:
        g[i] = 1

In [292]:
# Revenue parameters

# Load factor
LF = 0.8

# Create Yield matrix dict from formula in RPK using distance matrix
# Formula: Yield = 5.9 ∙ dij^(−0.76) + 0.043

yield_matrix = {}
for i in airport_list:
    yield_matrix[i] = {}
    for j in airport_list:
        if i == j:
            yield_matrix[i][j] = 0
        else:
            yield_matrix[i][j] = 5.9 * (distance[i][j] ** (-0.76)) + 0.043



In [293]:
# Cost parameters

# All aircraft are leased, and therefore a leasing cost needs to be accounted for. 
# The weekly leasing cost is a fixed amount depending on the type of aircraft

# Fuel cost formula
# CF_kij = CF_k ∙ f ∙ dij / 1.5
# Where 
# CF_kij = fuel cost for aircraft type k on route i-j [€]
# CF_k = fuel cost for aircraft type k [galon/km]
f  = 1.42 # fuel cost [€/galon]
# dij = distance between airport i and j [km]

# Time-based costs formula
# CT_kij = CT_k ∙ dij / V_k 
# Where
# CT_kij = total time-based cost for aircraft type k on route i-j [€]
# CT_k = total time-based cost for aircraft type k [€/h]
# dij = distance between airport i and j [km]
# V_k = cruise speed for aircraft type k [km/h]

# Variable costs formula
# Op_Cost_kij = CX_kij + CF_kij + CT_kij

# Fixed leg costs
# CX_k depends on the aircraft type and is a fixed cost per flight

# Create a dictionary with the Op_Cost for each aircraft type and airport using the aircraft type as the key and the rest of the columns as sub-dictionaries

Op_Cost = {}
for k in aircraft_types:
    Op_Cost[k] = {}
    for i in airport_list:
        Op_Cost[k][i] = {}
        for j in airport_list:
            Op_Cost[k][i][j] = aircrafts[k]['Operating_c'] + \
                                 aircrafts[k]['Fuel_c'] * f * distance[i][j] / 1.5 + \
                                 aircrafts[k]['Time_c'] * distance[i][j] / aircrafts[k]['Speed']
            # It should be noted that for flights departing or arriving at the hub airport the operating costs can be assumed to be 30% lower due to economies of scale
            if i == hub or j == hub:
                Op_Cost[k][i][j] = Op_Cost[k][i][j] * 0.7

In [294]:
import gurobipy as gp
from gurobipy import GRB

In [295]:
# Create model

m = gp.Model("Aircraft and network model")

In [296]:
# Create variables

# w_ij: flow from airport i to airport j that transfers at the hub
w = m.addVars(airport_list, airport_list, vtype=GRB.INTEGER, name="w") 

# x_ij: direct from airport i to airport j that does not transfer at the hub
x = m.addVars(airport_list, airport_list, vtype=GRB.INTEGER, name="x")

# z_kij: number of flights from airport i to airport j with aircraft k
z = m.addVars(aircraft_types, airport_list, airport_list, vtype=GRB.INTEGER, name="z")

# y_k: number of aircraft of type k
y = m.addVars(aircraft_types, vtype=GRB.INTEGER, name="y")
    

In [297]:
# y = {
#     'AC_1': 40,
#     'AC_2': 40,
#     'AC_3': 40
# }

In [298]:
print(yield_matrix['LIRF'])

{'LIRF': 0, 'EDDT': 0.06992760533821954, 'RJAA': 0.048409444242827726, 'EDDM': 0.08224233288312846, 'LFPG': 0.07171716951350784, 'SAEZ': 0.0479594972066314, 'CYYZ': 0.049977078823230145, 'MMMX': 0.048284660608056, 'CYVR': 0.048816393354816426, 'EDDF': 0.07491690456679825, 'KJFK': 0.050146019628108274, 'SBGL': 0.04875673597061972, 'LEMD': 0.06788745977129522, 'ZSPD': 0.04873672866660819, 'OMDB': 0.05313252555828846, 'LTFM': 0.06728557892472306, 'ZBAA': 0.04927368774495451, 'VIDP': 0.05098876368167407, 'EFRO': 0.056847961972560174, 'EHEH': 0.07002580717154067, 'EFHK': 0.059783774223934535, 'LDZA': 0.0926161307536247, 'EBBR': 0.07038112485453549, 'EIDW': 0.062081669675308995, 'LGAV': 0.07205000376849223, 'BIKF': 0.055470713046114305, 'EETN': 0.0603273004231626, 'LKPR': 0.07548614997834914, 'FACT': 0.04913650249565579, 'ESSA': 0.06109816716248087, 'EGPH': 0.06171919391709879, 'LFQQ': 0.0698966658534102, 'LPPT': 0.062455637808824266}


In [299]:
# Create objective function to maximize profit

# Revenue
# Rev1: direct revenue from direct flights (not transferring at the hub)
rev_1 = gp.quicksum((yield_matrix[i][j] * distance[i][j] * (x[i, j] + 0.9 * w[i, j])) for i in airport_list for j in airport_list)

# Rev2: revenue from connecting passengers
# Revenue generated by passengers connecting at the hub will be 10% lower
#rev_2 = 0.9 * gp.quicksum((yield_matrix[k][i][j] * w[i, j] * distance[i][j]) for k in aircraft_types for i in airport_list for j in airport_list)

# Cost
# Cost1: Fixed weekly leasing cost for all aircraft types
cost_1 = gp.quicksum((aircrafts[k]['Lease_c'] * y[k]) for k in aircraft_types)

# Cost2: Operational costs per flight from i to j
# Op_Cost_kij = CF_kij + CT_kij + CX_k (for all aircraft types k)
# Multiply Op_Cost_kij by the corresponding z_kij to get the total cost for all flights from i to j

cost_2 = gp.quicksum((Op_Cost[k][i][j] * z[k, i, j]) for k in aircraft_types for i in airport_list for j in airport_list)


# Objective function
# Full objective function with revenue and cost
m.setObjective(rev_1 - cost_1 - cost_2,GRB.MAXIMIZE)
m.update()

In [300]:
# Special hub conditions
# TAT for flights to the hub are 50% longer than the normal TAT for each aircraft type

# Add TAT to hub to the aircraft dictionary

for i in aircraft_types:
    aircrafts[i]['TAT_hub'] = aircrafts[i]['TAT'] * 0.5


In [301]:
# Add constraints

# Constraint 1: number of passengers from airport i to airport j
# x_ij + w_ij <= demand_ij (for all i and j)
m.addConstrs((x[i, j] + w[i, j] <= demand[i][j] for i in airport_list for j in airport_list), name="c1")

# Constraint 2: Transfer passengers are only if the hub is not the origin or destination
# w_ij <= demand_ij * g_i * g_j (for all i and j)
m.addConstrs((w[i, j] <= demand[i][j] * g[i] * g[j] for i in airport_list for j in airport_list), "c2")

# Constraint 3: capacity verification for each flight leg
# x_ij + sum(w_im * (1 - g_j) for all m) + sum(w_mj * (1 - g_i) for all m) <= sum(z_kij * s_k * LF for all k) (for all i and j)
m.addConstrs((x[i, j] + 
              gp.quicksum(w[i, m] * (1 - g[j]) for m in airport_list) + 
              gp.quicksum(w[m, j] * (1 - g[i]) for m in airport_list) <= 
              gp.quicksum(z[k, i, j] * aircrafts[k]['Seats'] * LF for k in aircraft_types) 
              for i in airport_list for j in airport_list), "c3")

# Constraint 4: same departing and arriving aircrafts per airport
# sum(z_kij) = sum(z_kji) (for all i and k)
m.addConstrs((gp.quicksum(z[k, i, j] for j in airport_list) == gp.quicksum(z[k, j, i] for j in airport_list) for i in airport_list for k in aircraft_types), "c4")


# Constraint 5: block time verification for each aircraft total
# we should add a TAT for only incoming to the hub of 50% of the normal TAT
# sum((dij / sp_k + TAT_k + (TAT_hub for all j != hub)) * z_kij for all i and j) <= BT_k * y_k (for all k)
m.addConstrs(((gp.quicksum((distance[i][j] / aircrafts[k]['Speed'] + aircrafts[k]['TAT'] / 60  +
                          (aircrafts[k]['TAT_hub'] * (1 - g[j]) / 60)) * 
                           z[k, i, j] for i in airport_list for j in airport_list) <= 
                           aircrafts[k]['BT'] * y[k] * 7) for k in aircraft_types), "c5")


# Define matrix a[k, i, j] based on aircraft range and runway length
a = {}
for k in aircraft_types:
    for i in airport_list:
        for j in airport_list:
            if (distance[i][j] <= aircrafts[k]['Range'] and \
                airports[i]['Runway (m)'] >= aircrafts[k]['Runway']) or i == j:
                a[k, i, j] = 100000 
            else:
                a[k, i, j] = 0

# Constraint 6: aircraft range verification
# z_kij <= a_kij (for all k, i, j)
m.addConstrs((z[k, i, j] <= a[k, i, j] for k in aircraft_types for i in airport_list for j in airport_list), "c6")

# Constraint 7: no self flights
# z_kij = 0 (for all k and i)
m.addConstrs((z[k, i, j] == 0 for k in aircraft_types for i in airport_list for j in airport_list if i == j), "c8")


# Update model
m.update()

In [302]:
# Optimize model
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6735 rows, 5448 columns and 22772 nonzeros
Model fingerprint: 0xd2dae8ab
Variable types: 0 continuous, 5448 integer (0 binary)
Coefficient statistics:
  Matrix range     [6e-01, 3e+02]
  Objective range  [2e+01, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+05]
Found heuristic solution: objective -0.0000000
Presolve removed 4894 rows and 1836 columns
Presolve time: 0.02s
Presolved: 1841 rows, 3612 columns, 11475 nonzeros
Variable types: 0 continuous, 3612 integer (0 binary)

Root relaxation: objective 2.856865e+05, 2392 iterations, 0.07 seconds (0.08 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 285686.465    0  133   -0

In [303]:
# Print the solutions as a pd.DataFrame for each aircraft type

# Total trips from i to j by aircraft type k
solution_z_k = []

for k in aircraft_types:
    solution_z = pd.DataFrame(columns=airport_list, index=airport_list)
    for v in m.getVars():
        if v.varName[0] == 'z' and v.varName[2:6] == k:
            solution_z.loc[v.varName[7:11], v.varName[12:16]] = v.x
            # Add the aircraft type to the dataframe
    print('Total flights for aircraft type: %s \n' % aircrafts[k]['AC_type'], solution_z, '\n')
    solution_z['Aircraft type'] = aircrafts[k]['AC_type']
    solution_z_k.append(solution_z)

# Make a dataframe from solution_z_k with a column for each aircraft type
solution_z_k = pd.concat(solution_z_k, axis=0)

Total flights for aircraft type: Regional Jet 
      LIRF EDDT RJAA EDDM LFPG SAEZ CYYZ MMMX CYVR EDDF  ... EIDW LGAV BIKF  \
LIRF  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
EDDT  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  ...  0.0  0.0  0.0   
RJAA  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
EDDM  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  ...  0.0  0.0  0.0   
LFPG  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
SAEZ  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
CYYZ  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
MMMX  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
CYVR  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
EDDF  0.0  1.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
KJFK  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
SBGL  0.0  0.0  

In [304]:
# Total flow trips from i to j x_ij 
solution_x = pd.DataFrame(columns=airport_list, index=airport_list)
solution_w = pd.DataFrame(columns=airport_list, index=airport_list)

for v in m.getVars():
    if v.varName[0] == 'x':
        solution_x.loc[v.varName[2:6], v.varName[7:11]] = v.x
    elif v.varName[0] == 'w':
        solution_w.loc[v.varName[2:6], v.varName[7:11]] = v.x

print('Total pax per OD \n', solution_x, '\n')
print('Total pax per OD \n', solution_w, '\n')

Total pax per OD 
        LIRF   EDDT   RJAA   EDDM   LFPG   SAEZ CYYZ   MMMX CYVR   EDDF  ...  \
LIRF    0.0  112.0  155.0   81.0  181.0   63.0  0.0  102.0  0.0   71.0  ...   
EDDT   94.0    0.0    0.0   56.0  120.0    0.0  0.0    0.0  0.0   56.0  ...   
RJAA  147.0    0.0    0.0    0.0    0.0    0.0  0.0    0.0  0.0    0.0  ...   
EDDM   78.0   56.0    0.0    0.0  120.0    0.0  0.0    0.0  0.0   56.0  ...   
LFPG  189.0  120.0    0.0  120.0    0.0    0.0  0.0    0.0  0.0  120.0  ...   
SAEZ   63.0    0.0    0.0    0.0    0.0    0.0  0.0    0.0  0.0    0.0  ...   
CYYZ    0.0    0.0    0.0    0.0    0.0    0.0  0.0    0.0  0.0    0.0  ...   
MMMX   92.0    0.0    0.0    0.0    0.0    0.0  0.0    0.0  0.0    0.0  ...   
CYVR    0.0    0.0    0.0    0.0    0.0    0.0  0.0    0.0  0.0    0.0  ...   
EDDF   86.0   56.0    0.0   56.0  176.0    0.0  0.0    0.0  0.0    0.0  ...   
KJFK  150.0    0.0    0.0    0.0    0.0    0.0  0.0  256.0  0.0    0.0  ...   
SBGL   89.0    0.0    0.0    0.0 

In [305]:
# Total number of aircraft of each type and utilization hours as a % of the total block time of each aircraft type
# Calculate total flown hours per aircraft type
total_flown_hours = {}
total_possible_hours = {}

for k in aircraft_types:
    total_flown_hours[k] = 0
    total_possible_hours[k] = aircrafts[k]['BT'] * y[k].x * 7
    for i in airports:
        for j in airports:
            if i != j:
                total_flown_hours[k] += (distance[i][j] / aircrafts[k]['Speed'] + aircrafts[k]['TAT'] / 60 + 
                                         (aircrafts[k]['TAT_hub'] * (1 - g[j]) / 60)) * z[k, i, j].x
    print('Number of %s: %g' % (aircrafts[k]['AC_type'], y[k].x))
    print('Utilization %0.2f' % (total_flown_hours[k] / total_possible_hours[k] * 100),'%')   



Number of Regional Jet: 1
Utilization 99.32 %
Number of Single Aisle Twin Engine Jet: 3
Utilization 99.98 %
Number of Twin aisle twin engine jet: 6
Utilization 100.00 %


In [341]:
# Export the solutions to a csv files with a long dataset format
solution_z_k.reset_index(inplace=True)
solution_z_k.rename(columns={'index': 'Origin'}, inplace=True)
solution_z_k.set_index(['Aircraft type', 'Origin'], inplace=True)

# Transform the solution_z_k dataframe to a long dataset format
solution_z_k_long = solution_z_k.stack().reset_index()
solution_z_k_long.columns = ['Aircraft type', 'Origin', 'Destination', 'Total_trips']

# Export the solution_z_long dataframe to a csv file
solution_z_k_long.to_csv('P1_Solutions/solution_trips.csv')

# Transform the solution_x_w dataframe to a long dataset format
solution_w_long = solution_w.stack().reset_index()
solution_w_long.columns = ['Origin', 'Destination', 'Total_pax']

# Transform the solution_x_w dataframe to a long dataset format
solution_x_long = solution_x.stack().reset_index()
solution_x_long.columns = ['Origin', 'Destination', 'Total_pax']


# # Export the solutions long dataframes to a csv file
solution_w_long.to_csv('P1_Solutions/solution_pax_transfers.csv')
solution_x_long.to_csv('P1_Solutions/solution_pax.csv')

In [379]:
solution_w_long_filtered = solution_w_long[solution_w_long['Total_pax'] > 0]
solution_w_long_filtered[solution_w_long_filtered['Destination'] == 'KJFK']

,Origin,Destination,Total_pax
472,OMDB,KJFK,75.0
505,LTFM,KJFK,191.0
802,LGAV,KJFK,107.0


In [377]:
# Create a OD dataframe for the airport pairs with origin and destination coordinates and total flights

results = airport_info[['City Name','ICAO Code', 'Latitude (deg)', 'Longitude (deg)']].merge(
    airport_info[['City Name','ICAO Code', 'Latitude (deg)', 'Longitude (deg)']], how='cross', suffixes=('_origin', '_destination')
    )

hub_point = Point(airport_coords[hub][1], airport_coords[hub][0])

# Creata a point and a line for each airport pair
results['Origin'] = results.apply(lambda row: Point(row['Longitude (deg)_origin'], row['Latitude (deg)_origin']), axis=1)
results['Destination'] = results.apply(lambda row: Point(row['Longitude (deg)_destination'], row['Latitude (deg)_destination']), axis=1)
results['Direct_flights'] = results.apply(lambda row: LineString([row['Origin'], row['Destination']]), axis=1)
results['Transfer_flights'] = results.apply(lambda row: LineString([row['Origin'], hub_point, row['Destination']]), axis=1)

# Add the total flights per aircraft type to the results dataframe
for k in aircraft_types:
    results['Total_flights_' + k] = solution_z_k_long[solution_z_k_long['Aircraft type'] == aircrafts[k]['AC_type']]['Total_trips'].values

# Add the total pax to the results dataframe
results['Direct_pax'] = solution_x_long['Total_pax']
results['Transfer_pax'] = 0

results.set_index(['ICAO Code_origin', 'ICAO Code_destination'], inplace=True)

# Add a new column with the transfer pax to the results dataframe 
for i, row in solution_w_long_filtered.iterrows():
    results.at[(row['Origin'], hub),'Transfer_pax'] += row['Total_pax']
    results.at[(hub, row['Destination']),'Transfer_pax'] += row['Total_pax']

# Reset the index of the results dataframe
results.reset_index(inplace=True)

# Drop columns that are not needed like coordinates
results.drop(columns=['Latitude (deg)_origin', 'Longitude (deg)_origin', 'Latitude (deg)_destination', 'Longitude (deg)_destination'], inplace=True)

# Drop the rows with same origin and destination
results.drop(results[results['ICAO Code_origin'] == results['ICAO Code_destination']].index, inplace=True)

# Create a column with the OD pair string, but if the pair is duplicated, the order is inverted
results['OD_pair'] = np.where(results['ICAO Code_origin'] < results['ICAO Code_destination'], 
                               results['ICAO Code_origin'] + '_' + results['ICAO Code_destination'], 
                               results['ICAO Code_destination'] + '_' + results['ICAO Code_origin'])


# Export the results dataframe to a csv file
results.to_csv('P1_Solutions/results.csv')

In [385]:
results['Total_pax'] = results['Direct_pax'] + results['Transfer_pax']
results[['OD_pair','Total_pax']][results['Transfer_pax'] > 0].sort_values(by='OD_pair', ascending=False)

,OD_pair,Total_pax
429,LIRF_ZSPD,512.0
13,LIRF_ZSPD,512.0
528,LIRF_ZBAA,512.0
16,LIRF_ZBAA,512.0
17,LIRF_VIDP,256.0
561,LIRF_VIDP,512.0
363,LIRF_SBGL,768.0
11,LIRF_SBGL,768.0
165,LIRF_SAEZ,512.0
5,LIRF_SAEZ,512.0
